深度学习框架（例如，MxNet、飞桨和PyTorch）会在后端自动构建计算图。

利用计算图，系统可以了解所有依赖关系，并且可以选择性地并行执行多个不相互依赖的任务以提高速度。例如， 12.2节中独立初始化两个变量。因此，系统可以选择并行执行它们。

通常情况下单个操作符将使用所有CPU或单个GPU上的所有计算资源。例如，即使在一台机器上有多个CPU处理器，dot操作符也将使用所有CPU上的所有核心（和线程）。这样的行为同样适用于单个GPU。因此，并行化对单设备计算机来说并不是很有用，而并行化对于多个设备就很重要了。虽然并行化通常应用在多个GPU之间，但增加本地CPU以后还将提高少许性能。例如， (Hadjis et al., 2016)则把结合GPU和CPU的训练应用到计算机视觉模型中。借助自动并行化框架的便利性，我们可以依靠几行Python代码实现相同的目标。对自动并行计算的讨论主要集中在使用CPU和GPU的并行计算上，以及计算和通信的并行化内容。

请注意，本节中的实验至少需要两个GPU来运行。

为了符合实验环境，可以在Kaggle的Note上运行，在使用代码前，通过`!pip install d2l`下载对应的库

如果下载失败，通过https://www.kaggle.com/datasets/lidailin05/d2l-module 中的文件，在Note右侧的Input处上传数据集，然后在开头添加模块搜索路径


从定义一个具有参考性的用于测试的工作负载开始：下面的run函数将执行10次矩阵－矩阵乘法时需要使用的数据分配到两个变量（x_gpu1和x_gpu2）中，这两个变量分别位于选择的不同设备上。

In [ ]:
import torch
from d2l import torch as d2l

In [ ]:
devices = d2l.try_all_gpus()
def run(x):
    return [x.mm(x) for _ in range(50)]

x_gpu1 = torch.rand(size=(4000, 4000), device=devices[0])
x_gpu2 = torch.rand(size=(4000, 4000), device=devices[1])

现在使用函数来处理数据。通过在测量之前需要预热设备（对设备执行一次传递）来确保缓存的作用不影响最终的结果。torch.cuda.synchronize()函数将会等待一个CUDA设备上的所有流中的所有核心的计算完成。函数接受一个device参数，代表是哪个设备需要同步。如果device参数是None（默认值），它将使用current_device()找出的当前设备。

预热过程<br>
为了排除初次运行时CUDA设备的启动、内存分配和缓存建立等开销，先调用一次run()函数对各个GPU（如x_gpu1和x_gpu2所在的设备）做一次前向传递。这样能够使设备进入稳定状态，确保后续的计时不会受到预热造成的不确定影响。

torch.cuda.synchronize()函数<br>
由于GPU操作默认是异步执行的，调用CUDA API后Python代码立即返回，但GPU任务可能还在后台队列中执行。<br>
torch.cuda.synchronize(device)函数用于等待指定设备上所有排队的CUDA任务完成。
- 如果传入特定设备（例如devices[0]或devices[1]），函数会同步该设备；
- 如果不指定device参数，则默认同步当前活跃设备。

分配到不同GPU
代码中利用d2l.try_all_gpus()获得所有可用GPU设备列表。随后，分别将不同的数据（x_gpu1、x_gpu2）分配到devices[0]和devices[1]上。在预热和计时时，分别调用torch.cuda.synchronize(devices[0])和torch.cuda.synchronize(devices[1])，以确保各自设备上的所有操作完成后才进行计时或结果收集。

这种设计确保了：
- GPU设备在正式测量前都已经就绪。
- 每个GPU的计时都是在其所有任务完成后进行，从而获得准确的运行时间。

In [ ]:
run(x_gpu1)
run(x_gpu2)  # 预热设备
torch.cuda.synchronize(devices[0])
torch.cuda.synchronize(devices[1])

with d2l.Benchmark('GPU1 time'):
    run(x_gpu1)
    torch.cuda.synchronize(devices[0])

with d2l.Benchmark('GPU2 time'):
    run(x_gpu2)
    torch.cuda.synchronize(devices[1])

如果删除两个任务之间的synchronize语句，系统就可以在两个设备上自动实现并行计算。

在with块内，先调用run(x_gpu1)向第一个GPU提交计算任务，再调用run(x_gpu2)向第二个GPU提交计算任务。由于GPU操作默认是异步的，这两组任务在各自对应的GPU上会并行运行，而CPU不会等待前一个任务完成后才提交下一个任务。

调用torch.cuda.synchronize()没有指定device参数，会同步当前活跃设备（或所有设备，具体取决于设置），确保所有GPU的任务都完成后再退出计时块，从而保证得到准确的并行计算时间。

因此，这段代码利用异步提交任务的特性，将两个GPU上的计算工作并行执行，并通过同步函数等待全部任务结束，从而实现了并行计算。

In [ ]:
with d2l.Benchmark('GPU1 & GPU2'):
    run(x_gpu1)
    run(x_gpu2)
    torch.cuda.synchronize()

在许多情况下，我们需要在不同的设备之间移动数据，比如在CPU和GPU之间，或者在不同的GPU之间。例如，当执行分布式优化时，就需要移动数据来聚合多个加速卡上的梯度。让我们通过在GPU上计算，然后将结果复制回CPU来模拟这个过程。

In [ ]:
def copy_to_cpu(x, non_blocking=False):
    return [y.to('cpu', non_blocking=non_blocking) for y in x]

# 测量的是在GPU上运行任务的时间。
with d2l.Benchmark('在GPU1上运行'):
    y = run(x_gpu1)
    torch.cuda.synchronize()

# 测量的是从GPU到CPU数据传输的时间。
with d2l.Benchmark('复制到CPU'):
    y_cpu = copy_to_cpu(y)
    torch.cuda.synchronize()

这种方式效率不高。注意到当列表中的其余部分还在计算时，我们可能就已经开始将y的部分复制到CPU了。例如，当计算一个小批量的（反传）梯度时。某些参数的梯度将比其他参数的梯度更早可用。因此，在GPU仍在运行时就开始使用PCI-Express总线带宽来移动数据是有利的。在PyTorch中，to()和copy_()等函数都允许显式的non_blocking参数，这允许在不需要同步时调用方可以绕过同步。设置non_blocking=True以模拟这个场景。

In [ ]:
with d2l.Benchmark('在GPU1上运行并复制到CPU'):
    y = run(x_gpu1)
    y_cpu = copy_to_cpu(y, True)
    torch.cuda.synchronize()

两个操作所需的总时间少于它们各部分操作所需时间的总和。请注意，与并行计算的区别是通信操作使用的资源：CPU和GPU之间的总线。事实上，我们可以在两个设备上同时进行计算和通信。如上所述，计算和通信之间存在的依赖关系是必须先计算y[i]，然后才能将其复制到CPU。幸运的是，系统可以在计算y[i]的同时复制y[i-1]，以减少总的运行时间。